In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [4]:
weather_df = pd.read_csv('output_data/weather_data.csv')
weather_df

,Country,City,Latitude,Long,Temperature(°F),Humidity %,Cloudiness %,Wind Speed(mph),Date
0,ZA,Port Alfred,-33.5906,26.8910,65.30,87,47,2.57,10/23/22 22:23:52
1,PE,Paita,-5.0892,-81.1144,62.28,85,98,11.48,10/23/22 22:32:08
2,EC,Puerto Ayora,-0.7393,-90.3518,66.54,88,81,5.99,10/23/22 22:24:49
3,SB,Kirakira,-10.4544,161.9205,81.37,82,71,6.08,10/23/22 22:24:58
4,ID,Denpasar,-8.6500,115.2167,85.71,79,40,10.36,10/23/22 22:28:24
...,...,...,...,...,...,...,...,...,...
551,ML,Taoudenni,22.6783,-3.9836,77.61,18,78,10.74,10/23/22 22:47:51
552,IN,Saiha,22.4833,92.9667,69.71,97,100,2.13,10/23/22 22:47:53
553,ID,Sampit,-2.5333,112.9500,87.17,62,100,5.44,10/23/22 22:47:54
554,CA,Port-Cartier,50.0334,-66.8654,44.28,89,90,3.11,10/23/22 22:47:56


In [5]:
gmaps.configure(api_key=g_key)

locations = weather_df[ ['Latitude','Long'] ].astype(float)
weather_df_drop = weather_df.dropna()
humidity = weather_df['Humidity %'].astype(float)

In [19]:
fig = gmaps.figure()
heat_map = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 250, point_radius = 2)
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

In [43]:
## Isolate the ideal conditions for snow. This is my ideal weather condition.

ideal_weather_df = weather_df.loc[(weather_df['Temperature(°F)'] <= 32) & (weather_df['Humidity %'] >= 60)]
ideal_weather_df

,Country,City,Latitude,Long,Temperature(°F),Humidity %,Cloudiness %,Wind Speed(mph),Date
9,CA,Tuktoyaktuk,69.4541,-133.0374,17.60,79,75,4.61,10/23/22 22:32:21
11,CA,Yellowknife,62.4560,-114.3525,28.92,93,100,6.91,10/23/22 22:23:45
15,CA,Thompson,55.7435,-97.8558,30.36,69,100,13.80,10/23/22 22:30:30
20,RU,Pevek,69.7008,170.3133,14.14,95,100,2.48,10/23/22 22:24:39
28,US,Barrow,71.2906,-156.7887,24.82,86,100,21.85,10/23/22 22:21:13
...,...,...,...,...,...,...,...,...,...
496,RU,Verkhoyansk,67.5447,133.3850,11.01,87,97,3.94,10/23/22 22:46:19
503,RU,Evensk,61.9500,159.2333,16.23,77,76,8.12,10/23/22 22:46:30
505,KZ,Līsakovsk,52.5369,62.4936,24.69,80,11,8.14,10/23/22 22:46:34
506,RU,Srednekolymsk,67.4500,153.6833,2.28,92,95,3.11,10/23/22 22:46:35


In [71]:
hotel_df = ideal_weather_df[['Country','City','Latitude','Long']].copy()
hotel_df

,Country,City,Latitude,Long
9,CA,Tuktoyaktuk,69.4541,-133.0374
11,CA,Yellowknife,62.4560,-114.3525
15,CA,Thompson,55.7435,-97.8558
20,RU,Pevek,69.7008,170.3133
28,US,Barrow,71.2906,-156.7887
...,...,...,...,...
496,RU,Verkhoyansk,67.5447,133.3850
503,RU,Evensk,61.9500,159.2333
505,KZ,Līsakovsk,52.5369,62.4936
506,RU,Srednekolymsk,67.4500,153.6833


In [72]:
params = {
    "key": g_key,
    "radius": 5000,
    'types':'hotel',
    'keyword': 'hotel'
         }

for index,row in hotel_df.iterrows():
    
    lat = row['Latitude']
    long = row['Long']
    params['location'] = f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels_request = requests.get(base_url, params=params)
    
    hotels_request = hotels_request.json()
    
    try:
        hotel_df.loc[index, "hotel_name"] = hotels_request["results"][0]["name"]
        print(f"Found Hotel for: {row['City']}")
    except (KeyError, IndexError):
        print(f"Hotel not found in {row['City']}! Too far!")

Found Hotel for: Tuktoyaktuk
Found Hotel for: Yellowknife
Found Hotel for: Thompson
Found Hotel for: Pevek
Found Hotel for: Barrow
Found Hotel for: Yar-Sale
Found Hotel for: Qaanaaq
Hotel not found in Provideniya! Too far!
Hotel not found in Yerbogachën! Too far!
Found Hotel for: Husavik
Found Hotel for: Norman Wells
Found Hotel for: Iqaluit
Hotel not found in Lavrentiya! Too far!
Hotel not found in Dikson! Too far!
Found Hotel for: Borovskoy
Hotel not found in Cherskiy! Too far!
Found Hotel for: Tiksi
Found Hotel for: Igarka
Hotel not found in Belaya Gora! Too far!
Hotel not found in Upernavik! Too far!
Found Hotel for: Canmore
Found Hotel for: Leningradskiy
Found Hotel for: Shaunavon
Found Hotel for: Khatanga
Hotel not found in Chokurdakh! Too far!
Found Hotel for: Rovaniemi
Hotel not found in Saskylakh! Too far!
Found Hotel for: Longyearbyen
Found Hotel for: Fairbanks
Found Hotel for: Esso
Hotel not found in Beringovskiy! Too far!
Hotel not found in Deputatsky! Too far!
Found Hotel 

In [87]:
hotel_df = hotel_df.dropna()
hotel_df = hotel_df.reset_index()
hotel_clean_df = hotel_df[ ['Country','City','hotel_name'] ].copy()
hotel_clean_df

,Country,City,hotel_name
0,CA,Tuktoyaktuk,Smitty's Bed and Breakfast
1,CA,Yellowknife,The Explorer Hotel
2,CA,Thompson,Best Western Thompson Hotel & Suites
3,RU,Pevek,Munitsipal'naya Gostinitsa
4,US,Barrow,Top of the World Hotel
5,RU,Yar-Sale,"Hotel ""Nord House"""
6,GL,Qaanaaq,Qaanaaq Hotel
7,IS,Husavik,Fosshotel Húsavík
8,CA,Norman Wells,Heritage Hotel
9,CA,Iqaluit,Aqsarniit Hotel & Conference Centre


In [91]:
info_box_content = [
    f"Country: {row['Country']}, City: {row['City']}, Hotel:{row['hotel_name']}" for index,row in hotel_clean_df.iterrows()
]


["Country: CA, City: Tuktoyaktuk, Hotel:Smitty's Bed and Breakfast",
 'Country: CA, City: Yellowknife, Hotel:The Explorer Hotel',
 'Country: CA, City: Thompson, Hotel:Best Western Thompson Hotel & Suites',
 "Country: RU, City: Pevek, Hotel:Munitsipal'naya Gostinitsa",
 'Country: US, City: Barrow, Hotel:Top of the World Hotel',
 'Country: RU, City: Yar-Sale, Hotel:Hotel "Nord House"',
 'Country: GL, City: Qaanaaq, Hotel:Qaanaaq Hotel',
 'Country: IS, City: Husavik, Hotel:Fosshotel Húsavík',
 'Country: CA, City: Norman Wells, Hotel:Heritage Hotel',
 'Country: CA, City: Iqaluit, Hotel:Aqsarniit Hotel & Conference Centre',
 'Country: KZ, City: Borovskoy, Hotel:Marzhan',
 'Country: RU, City: Tiksi, Hotel:Arktika',
 'Country: RU, City: Igarka, Hotel:Igarka Airport Hotel',
 'Country: CA, City: Canmore, Hotel:Canmore Inn & Suites',
 'Country: RU, City: Leningradskiy, Hotel:Gostinitsa',
 'Country: CA, City: Shaunavon, Hotel:Canalta Hotel Shaunavon',
 'Country: RU, City: Khatanga, Hotel:Gostinit

In [94]:
marker_locations = hotel_df[ ['Latitude','Long'] ]
fig = gmaps.figure()
markers = gmaps.marker_layer(
    marker_locations,
    info_box_content=info_box_content)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))